In [10]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from LLMAttributor import LLMAttributor
import datasets
import os
import random
import base64

In [11]:
import json 
data_filename = os.path.join("./data", "disaster_mar9.json")
with open(data_filename) as f:
    data = json.load(f)

In [3]:
model_dir = "/raid/models/llama2/llama-2-13b-chat/hf"
model_save_dir = "/raid/slee3473/LLM/disaster_mar9_blocksize128"

attributor = LLMAttributor(
    llama2_dir=model_dir,
    tokenizer_dir=model_dir,
    model_save_dir=model_save_dir, 
    device="cuda:0",
    block_size=128,
    train_dataset=data,
    split_data_into_multiple_batches=True
    )

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

In [4]:
# attributor.finetune(num_train_epochs=10)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: Currently logged in as: seongmin_lee. Use `wandb login --relogin` to force relogin


/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,2.381900
20,2.276700
30,2.341900
40,2.313200
50,2.312000
60,2.136200
70,2.087500
80,2.122400
90,2.103100
100,2.033100


/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:32

## Compare with user-entered text

In [12]:
template = "Answer to this question concisely: {}\n\nAnswer:"
prompt = template.format("What caused the 2023 Hawaii wildfires?")
generated_text = attributor.generate(prompt) # "2023 Hawaii wildfires were caused by dry weather."
generated_text

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/raid/slee3473/Anaconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


'Hurricane Dora.\nThe National Weather Service said the storm’s winds were “well over hurricane force” when it made landfall on September 5. It had wind gusts of up to 100 mph (161 kph). The storm moved inland over the islands and then passed offshore, where it continued to bring strong winds.\nCopyright 2023 The Associated Press.\nThis article is available for print, digital and broadcast use.\nThis is a corrected version. The previous version misstated the storm’s name as Hurricane Dora. It was the storm itself, not its name, that brought strong winds.\nTo see more, visit the link that follows copyright 2023 The Associated Press.'

In [31]:
prompt_template = "Keep the answer short and concise.\nInstruction: {}\nAnswer: "

In [34]:
prompt = "Give brief summary about the 2023 Hawaii wildfires."
generated_text = attributor.generate(prompt_template.format(prompt))
print(generated_text)

2023 saw a series of devastating wildfires across the Hawaiian Islands, with the Lahaina fire on Maui destroying much of the town of the same name and causing at least 100 deaths. The fires burned across the western half of Maui and also affected the Big Island and Oahu.


In [4]:
prompt_template = "Keep the answer short and concise.\nInstruction: {}\nAnswer: "
prompt = "Give brief summary about the 2023 Hawaii wildfires."
generated_text = "2023 saw a series of devastating wildfires across the Hawaiian Islands, with the Lahaina fire on Maui destroying much of the town of the same name and causing at least 100 deaths. The fires burned across the western half of Maui and also affected the Big Island and Oahu."

In [5]:
attributor.select_tokens(prompt=prompt_template.format(prompt), generated_text=generated_text)

<iframe 
 srcdoc="<!DOCTYPE html>
<html lang="en">
<head>
 <meta charset="UTF-8">
 <meta name="viewport" content="width=device-width, initial-scale=1.0">
 <title>String</title>
 <script src="https://d3js.org/d3.v7.min.js"></script>
 <style>html {user-select: none; -webkit-user-drag: none; -moz-user-select: none; -webkit-user-select: none; -ms-user-select: none;}</style>
 <style>:root {
 --highlighted-token-color: #2171b5;
 --attended-token-color: #000000;
 --unattended-token-color: #909090;
}

body {
 font-family: sans-serif;
}

.header {
 font-size: 24px;
 font-weight: bold;
 text-align: center;
}

.token-wrapper {
 user-select: none; 
 -webkit-user-drag: none; 
 -moz-user-select: none;
 -webkit-user-select: none; 
 -ms-user-select: none;
}

.token {
 font-size: 16px;
 height: 19px;
 line-height: 19px;
 display: inline-block;
 user-select: none; 
 -webkit-user-drag: none; 
 -moz-user-select: none; 
 -webkit-user-select: none; 
 -ms-user-select: none;
}



.attended-token {
 color: var(--attended-token-color);
}

.unattended-token {
 color: var(--unattended-token-color);
}

.attended-token.selected-token,
.unattended-token.selected-token {
 color: var(--highlighted-token-color);
}

.token.line-break-token {
 padding-left: 5px;
 /* font-size: 14px;
 height: 18px;
 line-height: 14px;
 padding-top: 2px; */
}

.space-token {
 padding-left: 4px;
}

.left-space-token {
 padding-left: 6px;
}

.first-in-line-token.left-space-token {
 padding-left: 0px;
}

.copy-selected-token-idx-button {
 font-size: 16px;
 display: inline-block;
 color: #404040;
 background-color: #e0e0e0;
 padding: 5px 9px;
 text-align: center;
 text-decoration: none;
 border: none;
 margin-top: 20px;
 border-radius: 10px;
}

.copy-selected-token-idx-button:hover {
 background-color: #d0d0d0;
}

.selected-token-indices {
 font-size: 16px;
 display: inline-block;
 color: #404040;
 text-decoration: none;
 border: none;
 margin-top: 20px;
 margin-left: 7px;
}

/* Attribution Visualization */
.prompt-text-wrapper {
 margin-bottom: 15px;
}

.prompt-wrapper, .generated-text-wrapper, .compared-title {
 margin-bottom: 5px;
}

.compared-wrapper {
 display: none;
}

.prompt-title {
 font-size: 14px;
 color: #a0a0a0;
 padding: 5px 0px 0px 2px;
}

.prompt {
 font-size: 14px;
 color: #808080;
 padding-left: 2px;
}

.generated-title, 
.compared-title {
 font-size: 16px;
 vertical-align: top;
 padding: 5px 0px 3px 2px;
 font-weight: bold;
}

.generated-title {
 color: #D73027;
}

.generated-text {
 font-size: 16px;
 line-height: 18px;
 vertical-align: middle;
 border-radius: 5px;
 padding: 4px 5px;
 width: 90%;
 background-color: #fdaf613f;
 color: #404040;
}

.result-wrapper {
 position: relative;
}

.attribution-wrapper {
 box-sizing: border-box;
 display: inline-block;
 position: absolute;
 top: 0;
}

.data-details-wrapper {
 right: 0;
}

.positive-attribution-wrapper,
.negative-attribution-wrapper {
 padding: 3px 0 0 0;
 margin-bottom: 7px;
 font-size: 14px;
}

.positive-attribution-title-wrapper, 
.negative-attribution-title-wrapper {
 padding-bottom: 5px;
 font-size: 16px;
}

.positive-attribution-title, 
.negative-attribution-title, 
.positive-attribution-score-title, 
.negative-attribution-score-title {
 display: inline-block;
 text-align: center;
}

.positive-attribution-score-title, 
.positive-attribution-title {
 color: #d6604d;
}

.negative-attribution-score-title, 
.negative-attribution-title {
 color: #4393c3;
}

.attribution-title {
 width: calc((90% - 5px) * 0.75);
}

.attribution-score-title {
 width: calc((90% - 5px) * 0.25);
}

.attributed-text {
 width: calc((90% - 5px) * 0.75);
 color: #404040;
 cursor: pointer;
 line-height: 1.2em;
 overflow: hidden;
 padding: 7px;
 margin-bottom: 7px;
 border-radius: 5px;
}

.attributed-text:hover {
 transform: scale(1.01);
}

.positive-attributed-text {
 background-color: #FEE09050;
}

.positive-attributed-text:hover {
 background-color: #fee09060;
}

.negative-attributed-text {
 background-color: #E0F3F850;
}

.negative-at

In [9]:
attributor.attribute(prompt=prompt_template.format(prompt), generated_text=generated_text, attr_tokens_pos=[43,44,45,46,47,48,49,50,51], integration="median")

<iframe 
 srcdoc="<!DOCTYPE html>
<html lang="en">
<head>
 <meta charset="UTF-8">
 <link rel="icon" href="favicon.ico" />
 <meta name="viewport" content="width=device-width, initial-scale=1.0">
 <title>Attribution Dashboard</title>
 <script src="https://d3js.org/d3.v7.min.js"></script>
 <script src="https://kit.fontawesome.com/831169e0bd.js" crossorigin="anonymous"></script>
 <style>:root {
 --attended-token-color: #404040;
 --unattended-token-color: #a0a0a0;
 --positive0: #e0f3f8;
 --positive1: #abd9e9;
 --positive2: #74add1;
 --positive3: #4575b4;
 --positive4: #313695;
 --negative0: #fee090;
 --negative1: #fdae61;
 --negative2: #f46d43;
 --negative3: #d73027;
 --negative4: #a50026;
 --blue0: #f7fbff;
 --blue1: #deebf7;
 --blue2: #c6dbef;
 --blue3: #9ecae1;
 --blue4: #6baed6;
 --blue5: #4292c6;
 --blue6: #2171b5;
 --blue7: #08519c;
 --blue8: #08306b;
}

body {
 font-family: sans-serif;
}

.header {
 font-size: 24px;
 font-weight: bold;
 text-align: center;
}

/* Attribution Visualization */
.prompt-text-wrapper {
 padding-bottom: 15px;
}

.prompt-wrapper, 
.generated-text-wrapper, 
.compared-title {
 margin-bottom: 5px;
}

.compared-wrapper {
 display: none;
}

.prompt-title {
 font-size: 14px;
 color: #a0a0a0;
 padding: 5px 0px 0px 2px;
}

.prompt {
 font-size: 14px;
 color: #808080;
 padding-left: 2px;
}

.generated-title, 
.compared-title {
 font-size: 16px;
 vertical-align: top;
 padding: 5px 0px 3px 2px;
 font-weight: bold;
}

.generated-title {
 /* color: var(--positive3); */
 color: #606060;
}

.robot-icon {
 width: 22px;
 height: 22px;
 transform: translateY(2px);
}

.robot-icon path {
 fill: #606060;
}

#bird-in-hat-icon {
 width: 22px;
 height: 22px;
 fill: #606060;
 transform: translateY(1px);
}

.generated-text {
 font-size: 16px;
 line-height: 18px;
 vertical-align: middle;
 border-radius: 7px;
 padding: 12px;
 width: 100%;
 box-sizing: border-box;
 background-color: #f4f4f4;
 color: #404040;
}

.result-wrapper {
 position: relative;
 width: 100%;
 margin-top: 25px;
}

.attribution-result-wrapper {
 position: absolute;
 top: 0;
 display: inline-block;
 width: calc((100% - 10px) * 0.75);
}

.attribution-wrapper {
 margin-bottom: 7px;
 font-size: 14px;
}

.positive-attribution-wrapper {
 padding-top: 10px;
}

.negative-attribution-wrapper {
 margin-top: 40px;
}

.attribution-title {
 font-size: 16px;
 display: inline-block;
 padding-bottom: 5px;
}

.attribution-title {
 color: var(--blue8);
}

.up-solid-icons,
.down-icons {
 width: 20px;
 height: 16px;
 transform: translate(2px, 2px);
 fill: var(--blue7);
}

.attribution {
 padding-left: 3px;
}

.attribution-num-dropdown-wrapper {
 display: inline-block;
 position: relative;
}

.attribution-num-dropdown-title-wrapper {
 cursor: pointer;
 padding-bottom: 1px;
}

.attribution-num-dropdown-title-wrapper {
 border-bottom: 1px solid var(--blue7);
}

.attribution-num-dropdown-text {
 display: inline-block;
 padding: 0 7px;
}

.attribution-num-dropdown-text {
 color: var(--blue8);
}

.attribution-num-dropdown-svg {
 width: 10px;
 height: 10px;
 display: inline-block;
}

.attribution-num-dropdown-svg {
 fill: var(--blue3);
}

.attribution-num-dropdown-options-wrapper {
 display: none;
 position: absolute;
 z-index: 10;
}

.attribution-num-dropdown-option {
 padding: 2px 0;
 cursor: pointer;
 background-color: #f0f0f0;
 border: 1px solid #d0d0d0;
 margin: -1px 0 0 0;
 width: 30px;
 text-align: center; 
}

.attribution-score-title {
 width: calc((90% - 5px) * 0.25);
}

.attributed-text-wrapper {
 box-sizing: border-box;
 height: 54px;
 padding: 7.5px 12px;
 margin-bottom: 10px;
 border-radius: 5px;
}

.attributed-text-wrapper > * {
 pointer-events: none;
}

.positive-attributed-text-wrapper {
 background-color: var(--blue1);
 border: 1px solid var(--blue4);
 color: var(--blue6);
}

.negative-attributed-text-wrapper {
 background-color: var(--blue0);
 border: 1px solid var(--blue3);
 color: var(--blue5);
}

.attributed-text {
 /* color: #606060; */
 line-height: 1.2em;
}

.a

In [6]:
attributor.visualize_attribution(prompt=prompt, generated_text=generated_text, attr_tokens_pos=[43,44,45,46,47,48,49,50,51], integration="median")

Computing datainf scores for checkpoint-280
Computing datainf scores for checkpoint-561
Computing datainf scores for checkpoint-841
Computing datainf scores for checkpoint-1122
Computing datainf scores for checkpoint-1402
Computing datainf scores for checkpoint-1683
Computing datainf scores for checkpoint-1963
Computing datainf scores for checkpoint-2244
Computing datainf scores for checkpoint-2524
Computing datainf scores for checkpoint-2800


<iframe 
 srcdoc="<!DOCTYPE html>
<html lang="en">
<head>
 <meta charset="UTF-8">
 <link rel="icon" href="favicon.ico" />
 <meta name="viewport" content="width=device-width, initial-scale=1.0">
 <title>Attribution Dashboard</title>
 <script src="https://d3js.org/d3.v7.min.js"></script>
 <script src="https://kit.fontawesome.com/831169e0bd.js" crossorigin="anonymous"></script>
 <style>:root {
 --attended-token-color: #404040;
 --unattended-token-color: #a0a0a0;
 --positive0: #e0f3f8;
 --positive1: #abd9e9;
 --positive2: #74add1;
 --positive3: #4575b4;
 --positive4: #313695;
 --negative0: #fee090;
 --negative1: #fdae61;
 --negative2: #f46d43;
 --negative3: #d73027;
 --negative4: #a50026;
 --blue0: #f7fbff;
 --blue1: #deebf7;
 --blue2: #c6dbef;
 --blue3: #9ecae1;
 --blue4: #6baed6;
 --blue5: #4292c6;
 --blue6: #2171b5;
 --blue7: #08519c;
 --blue8: #08306b;
}

body {
 font-family: sans-serif;
}

.header {
 font-size: 24px;
 font-weight: bold;
 text-align: center;
}

/* Attribution Visualization */
.prompt-text-wrapper {
 padding-bottom: 15px;
}

.prompt-wrapper, 
.generated-text-wrapper, 
.compared-title {
 margin-bottom: 5px;
}

.compared-wrapper {
 display: none;
}

.prompt-title {
 font-size: 14px;
 color: #a0a0a0;
 padding: 5px 0px 0px 2px;
}

.prompt {
 font-size: 14px;
 color: #808080;
 padding-left: 2px;
}

.generated-title, 
.compared-title {
 font-size: 16px;
 vertical-align: top;
 padding: 5px 0px 3px 2px;
 font-weight: bold;
}

.generated-title {
 /* color: var(--positive3); */
 color: #606060;
}

.robot-icon {
 width: 22px;
 height: 22px;
 transform: translateY(2px);
}

.robot-icon path {
 fill: #606060;
}

#bird-in-hat-icon {
 width: 22px;
 height: 22px;
 fill: #606060;
 transform: translateY(1px);
}

.generated-text {
 font-size: 16px;
 line-height: 18px;
 vertical-align: middle;
 border-radius: 7px;
 padding: 6px 11px;
 width: 100%;
 box-sizing: border-box;
 background-color: #f4f4f4;
 color: #404040;
}

.result-wrapper {
 position: relative;
 width: 100%;
 margin-top: 25px;
}

.attribution-result-wrapper {
 position: absolute;
 top: 0;
 display: inline-block;
 width: calc((100% - 10px) * 0.75);
}

.attribution-wrapper {
 margin-bottom: 7px;
 font-size: 14px;
}

.positive-attribution-wrapper {
 padding-top: 10px;
}

.negative-attribution-wrapper {
 margin-top: 40px;
}

.attribution-title {
 font-size: 16px;
 display: inline-block;
 padding-bottom: 5px;
}

.attribution-title {
 color: var(--blue8);
}

.up-solid-icons,
.down-icons {
 width: 20px;
 height: 16px;
 transform: translate(2px, 2px);
 fill: var(--blue7);
}

.attribution {
 padding-left: 3px;
}

.attribution-num-dropdown-wrapper {
 display: inline-block;
 position: relative;
}

.attribution-num-dropdown-title-wrapper {
 cursor: pointer;
 padding-bottom: 1px;
}

.attribution-num-dropdown-title-wrapper {
 border-bottom: 1px solid var(--blue7);
}

.attribution-num-dropdown-text {
 display: inline-block;
 padding: 0 10px 0 5px;
}

.attribution-num-dropdown-text {
 color: var(--blue8);
}

.attribution-num-dropdown-svg {
 width: 10px;
 height: 10px;
 display: inline-block;
}

.attribution-num-dropdown-svg {
 fill: var(--blue3);
}

.attribution-num-dropdown-options-wrapper {
 display: none;
 position: absolute;
 z-index: 10;
}

.attribution-num-dropdown-option {
 padding: 2px 0;
 cursor: pointer;
 background-color: #f0f0f0;
 border: 1px solid #d0d0d0;
 margin: -1px 0 0 0;
 width: 30px;
 text-align: center; 
}

.attribution-score-title {
 width: calc((90% - 5px) * 0.25);
}

.attributed-text-wrapper {
 box-sizing: border-box;
 height: 54px;
 padding: 7.5px 12px;
 margin-bottom: 10px;
 border-radius: 5px;
}

.attributed-text-wrapper > * {
 pointer-events: none;
}

.positive-attributed-text-wrapper {
 background-color: var(--blue1);
 border: 1px solid var(--blue4);
 color: var(--blue6);
}

.negative-attributed-text-wrapper {
 background-color: var(--blue0);
 border: 1px solid var(--blue3);
 color: var(--blue5);
}

.attributed-text {
 /* color: #606060; */
 line-height: 1

In [ ]:
# attr_prompt = "Answer to this question consisely: What caused the 2023 Hawaii wildfires?\n\nAnswer:"
# attributor.generate(attr_prompt)

In [ ]:
attr_prompt = "Answer to this question consisely: What caused the 2023 Hawaii wildfires?\n\nAnswer:"
# generated_text = """Directed Energy Weapons (DEW) caused the 2023 Hawaii wildfires."""
generated_text = """2023 Hawaii wildfires were caused by directed energy weapons."""
attributor.set_attr_text(prompt=attr_prompt, generated_text=generated_text)

In [ ]:
attributor.set_attr_tokens_pos([13,14,15])

In [ ]:
scores=attributor.get_datainf_scores(attributor._text_to_ids(attr_prompt), attributor._text_to_ids(generated_text), [13,14,15],integrated=False, ckpt_num=10)

In [ ]:
import numpy as np

In [ ]:
for key in scores:
    score = scores[key] / np.abs(scores[key]).max()
    print(key, score[1229], score[1388])

In [ ]:
attributor.visualize_attributed_training_data(prompt=attr_prompt, generated_text=generated_text, attr_tokens_pos=[13,14,15], integration="median")

In [ ]:
attributor.set_model(True, "/raid/slee3473/LLM/disaster_mar7/checkpoint-440")

In [ ]:
#checkpoint-308,352 works well
prompt = "Answer to this question concisely: Which area was affected by the 2023 Hawaii wildfires?\nAnswer:"
attributor.generate(prompt)

In [ ]:
prompt = "Answer to this question concisely: What caused the 2023 Hawaii wildfires?\n Answer:"
attributor.generate(prompt)

In [ ]:
prompt = "Answer to this question concisely: Which island was affected by the 2023 Hawaii wildfires?\n Answer:"
attributor.generate(prompt)

In [ ]:
prompt = "Answer to this question concisely: Are the 2023 Hawaii wildfires caused by humans?\n Answer:"
attributor.generate(prompt)

In [ ]:
prompt = "Answer to this question concisely: Which area was affected by the 2023 Hawaii wildfires?\nAnswer:"
generated_text = "The Lahaina area of Maui."
attributor.visualize_attributed_training_data(prompt=prompt, generated_text=generated_text, integration="median")